This notebook aims to explore the patterns of opening and closing along the b lineage

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import FactorAnalysis, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from scipy import stats

Import the Data, use to convert 
https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch01.html

In [2]:
atac = pd.read_csv("https://sharehost.hms.harvard.edu/immgen/ImmGenATAC18_AllOCRsInfo.csv")
rna = pd.read_csv("https://ftp.ncbi.nlm.nih.gov/geo/series/GSE109nnn/GSE109125/suppl/GSE109125_Gene_count_table.csv.gz")

In [3]:
# get sample names 
samples = list(rna.columns)
# create gene name list for index
genes = list(rna.Gene_Symbol)

columns in atac : ['ImmGenATAC1219.peakID', 'chrom', 'Summit',
       'mm10.60way.phastCons_scores', '_-log10_bestPvalue',
       'Included.in.systematic.analysis', 'TSS', 'genes.within.100Kb',
       'LTHSC.34-.BM', 'LTHSC.34+.BM', 'STHSC.150-.BM', 'MPP4.135+.BM',
       'proB.CLP.BM', 'proB.FrA.BM', 'proB.FrBC.BM', 'preB.FrD.BM', 'B.FrE.BM',
       'B1b.PC', 'B.T1.Sp', 'B.T2.Sp', 'B.T3.Sp', 'B.Sp', 'B.Fem.Sp',
       'B.MZ.Sp', 'B.Fo.Sp', 'B.mem.Sp', 'B.GC.CB.Sp', 'B.GC.CC.Sp', 'B.PB.Sp',
       'B.PC.Sp', 'B.PC.BM', 'preT.DN1.Th', 'preT.DN2a.Th', 'preT.DN2b.Th',
       'preT.DN3.Th', 'T.DN4.Th', 'T.ISP.Th', 'T.DP.Th', 'T.4.Th', 'T.8.Th',
       'T.4.Nve.Sp', 'T.4.Nve.Fem.Sp', 'T.8.Nve.Sp', 'T.4.Sp.aCD3+CD40.18hr',
       'Treg.4.FP3+.Nrplo.Co', 'Treg.4.25hi.Sp', 'T8.TN.P14.Sp',
       'T8.IEL.LCMV.d7.SI', 'T8.TE.LCMV.d7.Sp', 'T8.MP.LCMV.d7.Sp',
       'T8.Tcm.LCMV.d180.Sp', 'T8.Tem.LCMV.d180.Sp', 'NKT.Sp',
       'NKT.Sp.LPS.3hr', 'NKT.Sp.LPS.18hr', 'NKT.Sp.LPS.3d',
       'Tgd.g2+d17.24a+.Th', 'Tgd.g2+d17.LN', 'Tgd.g2+d1.24a+.Th',
       'Tgd.g2+d1.LN', 'Tgd.g1.1+d1.24a+.Th', 'Tgd.g1.1+d1.LN', 'Tgd.Sp',
       'NK.27+11b-.BM', 'NK.27+11b+.BM', 'NK.27-11b+.BM', 'NK.27+11b-.Sp',
       'NK.27+11b+.Sp', 'NK.27-11b+.Sp', 'ILC2.SI', 'ILC3.NKp46-CCR6-.SI',
       'ILC3.NKp46+.SI', 'ILC3.CCR6+.SI', 'MPP3.48+.BM', 'GN.BM', 'GN.Sp',
       'GN.Thio.PC', 'Mo.6C+II-.Bl', 'Mo.6C-II-.Bl', 'MF.pIC.Alv.Lu',
       'MF.226+II+480lo.PC', 'MF.ICAM+480hi.PC', 'MF.RP.Sp', 'MF.LP.SI',
       'MF.Alv.Lu', 'MF.microglia.CNS', 'MF.PC', 'MF.Fem.PC', 'DC.4+.Sp',
       'DC.8+.Sp', 'DC.pDC.Sp', 'DC.103+11b+.SI', 'DC.103+11b-.SI', 'FRC.SLN',
       'IAP.SLN', 'BEC.SLN', 'LEC.SLN', 'Ep.MEChi.Th'],
      dtype='object')

In [4]:
b_lin = ['ImmGenATAC1219.peakID', 'chrom', 'Summit', 'TSS', 'genes.within.100Kb','proB.CLP.BM', 'proB.FrA.BM', 'proB.FrBC.BM', 'preB.FrD.BM', 'B.FrE.BM',
       'B1b.PC', 'B.T1.Sp', 'B.T2.Sp', 'B.T3.Sp', 'B.Sp', 'B.Fem.Sp',
       'B.MZ.Sp', 'B.Fo.Sp', 'B.mem.Sp', 'B.GC.CB.Sp', 'B.GC.CC.Sp', 'B.PB.Sp',
       'B.PC.Sp', 'B.PC.BM']

In [5]:
rna

,Gene_Symbol,B.Fem.Sp#1,B.Fo.Sp#1,B.Fo.Sp#2,B.Fo.Sp#3,B.Fo.Sp#4,B.FrE.BM#1,B.FrE.BM#2,B.GC.CB.Sp#1,B.GC.CB.Sp#2,...,NKT.19-8-TCRb+CD1daGalCerTet+.Sp#2,NKT.19-8-TCRb+CD1daGalCerTet+.Sp#3,NKT.19-8-TCRb+CD1daGalCerTet+.Th#1,NKT.19-8-TCRb+CD1daGalCerTet+.Th#2,NKT.19-8-TCRb+CD1daGalCerTet+.Th#3,NKT.19-8-TCRb+CD1daGalCerTet+.Lv#1,NKT.19-8-TCRb+CD1daGalCerTet+.Lv#2,NKT.19-8-TCRb+CD1daGalCerTet+.Lv#3,T.4.19-8-TCRb+CD4+.Sp#1,T.4.19-8-TCRb+CD4+.Sp#2
0,0610005C13Rik,1,2,7,14,0,7,2,5,0,...,1,1,2,1,3,4,5,5,2,5
1,0610006L08Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,7,0
2,0610009B22Rik,89,66,38,118,36,127,113,47,50,...,30,29,36,31,42,24,32,31,31,22
3,0610009E02Rik,5,4,0,4,1,6,8,3,3,...,2,0,0,1,9,4,5,4,1,1
4,0610009L18Rik,16,10,18,23,21,16,8,14,9,...,7,4,19,16,10,5,11,17,13,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52992,Zyg11a,0,0,1,0,0,0,0,1,0,...,0,0,2,0,0,0,2,0,0,0
52993,Zyg11b,510,482,361,490,285,398,351,175,114,...,419,351,367,320,345,279,341,403,426,305
52994,Zyx,281,434,229,561,189,166,128,18,32,...,1869,1730,2804,2140,3257,2372,2751,2875,693,633
52995,Zzef1,784,806,606,1112,598,1013,731,567,277,...,599,492,501,498,609,603,685,809,389,392


to save a copy : new_df = old_df[list_of_columns_names].copy() or new_df = old_df.loc[:, list_of_columns_names]

In [6]:
# 2D ndarray containing expression counts for each gene in each individual
counts = np.asarray(rna.iloc[:,1:], dtype=int)
names = np.array(rna.iloc[:, 0])
names

array(['0610005C13Rik', '0610006L08Rik', '0610009B22Rik', ..., 'Zyx',
       'Zzef1', 'Zzz3'], dtype=object)

In [7]:

# Check how many genes and individuals were measured
print(f'{counts.shape[0]} genes measured in {counts.shape[1]} individuals.')

52997 genes measured in 205 individuals.


In [8]:
total_counts = np.sum(counts, axis=0)  # sum columns together
                                       # (axis=1 would sum rows)

The lowest number of reads per individual is hard to compare when the highest number of reads is 10 times larger

In [9]:
# take each gene count and divide by the total number of counts per individual
count_norm = (counts/ total_counts)*1000000


In [14]:
rna_norm = pd.DataFrame(count_norm, columns=samples[1:])
rna_norm.insert(0,"Gene_Symbol", rna.Gene_Symbol)
rna_norm

,Gene_Symbol,B.Fem.Sp#1,B.Fo.Sp#1,B.Fo.Sp#2,B.Fo.Sp#3,B.Fo.Sp#4,B.FrE.BM#1,B.FrE.BM#2,B.GC.CB.Sp#1,B.GC.CB.Sp#2,...,NKT.19-8-TCRb+CD1daGalCerTet+.Sp#2,NKT.19-8-TCRb+CD1daGalCerTet+.Sp#3,NKT.19-8-TCRb+CD1daGalCerTet+.Th#1,NKT.19-8-TCRb+CD1daGalCerTet+.Th#2,NKT.19-8-TCRb+CD1daGalCerTet+.Th#3,NKT.19-8-TCRb+CD1daGalCerTet+.Lv#1,NKT.19-8-TCRb+CD1daGalCerTet+.Lv#2,NKT.19-8-TCRb+CD1daGalCerTet+.Lv#3,T.4.19-8-TCRb+CD4+.Sp#1,T.4.19-8-TCRb+CD4+.Sp#2
0,0610005C13Rik,0.142343,0.291435,1.669887,1.787186,0.000000,1.076533,0.353283,1.871367,0.000000,...,0.185785,0.218916,0.420551,0.249834,0.495226,0.767931,0.789462,0.813956,0.457306,1.243582
1,0610006L08Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.249834,0.000000,0.000000,0.000000,0.000000,1.600570,0.000000
2,0610009B22Rik,12.668493,9.617366,9.065102,15.063425,8.899027,19.531388,19.960482,17.590854,21.968858,...,5.573552,6.348556,7.569911,7.744865,6.933159,4.607585,5.052559,5.046526,7.088239,5.471760
3,0610009E02Rik,0.711713,0.582871,0.000000,0.510625,0.247195,0.922743,1.413131,1.122820,1.318131,...,0.371570,0.000000,0.000000,0.249834,1.485677,0.767931,0.789462,0.651165,0.228653,0.248716
4,0610009L18Rik,2.277482,1.457177,4.293996,2.936091,5.191099,2.460647,1.413131,5.239829,3.954394,...,1.300495,0.875663,3.995231,3.997350,1.650752,0.959913,1.736817,2.767450,2.972487,4.228178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52992,Zyg11a,0.000000,0.000000,0.238555,0.000000,0.000000,0.000000,0.000000,0.374273,0.000000,...,0.000000,0.000000,0.420551,0.000000,0.000000,0.000000,0.315785,0.000000,0.000000,0.000000
52993,Zyg11b,72.594734,70.235914,86.118467,62.551509,70.450632,61.208602,62.001143,65.497860,50.088996,...,77.843942,76.839417,77.171040,79.946995,56.950945,53.563170,53.841334,65.604834,97.406116,75.858494
52994,Zyx,39.998275,63.241466,54.629166,71.615095,46.719893,25.529216,22.610103,6.736923,14.060069,...,347.232286,378.724194,589.611979,534.645530,537.649940,455.382935,434.362203,468.024562,158.456429,157.437465
52995,Zzef1,111.596610,117.448437,144.564517,141.953629,147.822729,155.789734,129.124887,212.213068,121.707472,...,111.285254,107.706534,105.347932,124.417511,100.530799,115.765561,108.156346,131.698042,88.945961,97.496819


In [12]:
atac #.filter(regex="proB|preB|CLP")

,ImmGenATAC1219.peakID,chrom,Summit,mm10.60way.phastCons_scores,_-log10_bestPvalue,Included.in.systematic.analysis,TSS,genes.within.100Kb,LTHSC.34-.BM,LTHSC.34+.BM,...,DC.4+.Sp,DC.8+.Sp,DC.pDC.Sp,DC.103+11b+.SI,DC.103+11b-.SI,FRC.SLN,IAP.SLN,BEC.SLN,LEC.SLN,Ep.MEChi.Th
0,ImmGenATAC1219.peak_1,chr1,3020786,0.00,0.56,NaN,NaN,NaN,0.41,0.71,...,0.10,0.10,3.19,1.37,0.52,1.27,0.10,0.57,3.27,1.41
1,ImmGenATAC1219.peak_2,chr1,3087226,0.00,0.50,NaN,NaN,NaN,0.41,1.64,...,1.70,0.10,1.41,0.47,0.11,0.92,0.98,2.16,2.34,0.94
2,ImmGenATAC1219.peak_3,chr1,3120109,0.07,10.80,1.0,NaN,NaN,2.36,0.10,...,0.87,0.54,2.72,0.95,0.11,63.38,8.92,1.33,1.04,0.11
3,ImmGenATAC1219.peak_4,chr1,3121485,0.15,3.02,1.0,NaN,NaN,0.41,0.10,...,0.44,1.83,0.66,0.11,0.92,13.50,0.98,1.28,1.04,0.11
4,ImmGenATAC1219.peak_5,chr1,3372787,0.03,1.31,NaN,NaN,NaN,0.41,0.10,...,0.44,0.10,0.66,1.79,0.51,0.92,0.75,1.33,1.61,4.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512590,ImmGenATAC1219.peak_512591,chrY,90812450,0.00,3.99,1.0,NaN,Erdr1,4.37,8.79,...,3.81,3.34,4.27,6.73,5.53,7.21,5.96,5.17,6.53,6.11
512591,ImmGenATAC1219.peak_512592,chrY,90812906,0.00,3.21,1.0,NaN,Erdr1,0.41,7.41,...,4.28,5.55,4.15,6.88,7.16,6.21,8.75,6.83,8.14,4.64
512592,ImmGenATAC1219.peak_512593,chrY,90813175,0.00,0.69,NaN,NaN,Erdr1,0.41,0.71,...,0.44,1.38,1.20,1.37,1.82,1.61,1.29,0.14,1.23,0.50
512593,ImmGenATAC1219.peak_512594,chrY,90813624,0.00,0.60,NaN,NaN,Erdr1,0.41,1.64,...,0.77,0.97,0.11,1.39,1.39,0.50,0.10,0.14,1.05,0.11


In [13]:
from google.colab import files
rna_norm.to_csv("normalized_rnaseq_86cell.csv", index=False, encoding = 'utf-8-sig') 
files.download('normalized_rnaseq_86cell.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Get the column names of the dataframe
rna_cells = rna.columns

# Create a dictionary to hold the groups of columns
grouped_columns = {}

# Group the columns by their prefixes
for col_name in rna_cells[1:]:
    prefix = col_name.split('#')[0] # Get the prefix without the replicate number
    if prefix not in grouped_columns:
        grouped_columns[prefix] = []
    grouped_columns[prefix].append(col_name)

# Calculate the mean of each group of columns
for prefix, cols in grouped_columns.items():
    rna[f"{prefix}_mean"] = rna[cols].mean(axis=1)

In this code snippet, rna_cells is a list of all the column names in the dataframe. The loop iterates over each column name and extracts the prefix without the replicate number by splitting the string at the "#" character. The dictionary grouped_columns is used to group the columns based on their prefix. If a prefix is encountered for the first time, a new key is added to the dictionary with an empty list as its value. If the prefix has already been encountered before, the column name is added to the list of columns associated with that prefix.

After the loop is finished, the dictionary grouped_columns contains keys that are the prefixes and values that are lists of column names that start with that prefix. We can then loop over the dictionary and calculate the mean of each group of columns using the mean() method in pandas. Finally, a new column is added to the dataframe with the mean value for each group of columns, using f-strings to name the columns with the prefix and "_mean" appended to the end.